# Multi-Step AI Pipeline Execution

This notebook executes complex multi-step AI reasoning chains using OpenCode CLI.

**Parameters:**
- `pipeline_file`: YAML file defining the pipeline steps
- `output_dir`: Directory for storing intermediate and final results
- `session_id`: Optional session ID for continuity (auto-generated if not provided)

In [ ]:
# Default parameters - will be overridden by papermill
pipeline_file = "pipeline_research.yaml"
output_dir = "output"
session_id = ""

In [ ]:
#!/bin/bash

# Setup pipeline execution environment
echo "=== Multi-Step AI Pipeline Execution ==="
echo "Pipeline file: $pipeline_file"
echo "Output directory: $output_dir"
echo "Session ID: $session_id"
echo

# Create output directory with timestamp
TIMESTAMP=$(date '+%Y%m%d_%H%M%S')
if [ -z "$session_id" ]; then
    session_id="pipeline_${TIMESTAMP}"
fi

FULL_OUTPUT_DIR="${output_dir}/${session_id}"
mkdir -p "$FULL_OUTPUT_DIR"

echo "Created output directory: $FULL_OUTPUT_DIR"
echo "Session ID: $session_id"
echo

In [ ]:
#!/bin/bash

# Load and validate pipeline configuration
echo "=== Loading Pipeline Configuration ==="

if [ ! -f "$pipeline_file" ]; then
    echo "ERROR: Pipeline file '$pipeline_file' not found!"
    exit 1
fi

# Display pipeline configuration
echo "Pipeline configuration:"
cat "$pipeline_file"
echo

# Extract pipeline metadata using grep/sed (basic YAML parsing)
PIPELINE_NAME=$(grep "name:" "$pipeline_file" | head -1 | sed 's/.*name: *"\?//' | sed 's/"\?$//')
PIPELINE_DESC=$(grep "description:" "$pipeline_file" | head -1 | sed 's/.*description: *"\?//' | sed 's/"\?$//')

echo "Pipeline Name: $PIPELINE_NAME"
echo "Description: $PIPELINE_DESC"
echo

# Save pipeline metadata
echo "# Pipeline Execution Report" > "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Pipeline:** $PIPELINE_NAME" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Description:** $PIPELINE_DESC" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Session ID:** $session_id" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Timestamp:** $(date)" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"

In [ ]:
#!/bin/bash

# Execute Step 1: Extract and run first step
echo "=== Executing Step 1 ==="

# Extract step 1 details (basic YAML parsing with sed/awk)
STEP1_NAME=$(awk '/- name:/{print $3; exit}' "$pipeline_file" | tr -d '"')
STEP1_PROMPT=$(awk '/- name:/,/- name:|^pipeline:/{if(/prompt:/) {sub(/.*prompt: *"?/,""); sub(/"?$/,""); print; exit}}' "$pipeline_file")
STEP1_MODEL=$(awk '/- name:/,/- name:|^pipeline:/{if(/model:/) {sub(/.*model: *"?/,""); sub(/"?$/,""); print; exit}}' "$pipeline_file")

echo "Step Name: $STEP1_NAME"
echo "Model: $STEP1_MODEL"
echo "Prompt: $STEP1_PROMPT"
echo

# Execute step 1
STEP1_OUTPUT="$FULL_OUTPUT_DIR/step1_${STEP1_NAME}.md"
echo "Executing OpenCode for step 1..."

if opencode "$STEP1_PROMPT" --model "$STEP1_MODEL" > "$STEP1_OUTPUT" 2>&1; then
    echo "✅ Step 1 completed successfully"
    echo "Output saved to: $STEP1_OUTPUT"
    
    # Add to pipeline report
    echo "## Step 1: $STEP1_NAME" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Status:** ✅ Completed" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Model:** $STEP1_MODEL" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Output:** [step1_${STEP1_NAME}.md](step1_${STEP1_NAME}.md)" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    
    # Show preview
    echo "--- Step 1 Output Preview ---"
    head -20 "$STEP1_OUTPUT"
    echo "--- End Preview ---"
else
    echo "❌ Step 1 failed"
    echo "Error output:"
    cat "$STEP1_OUTPUT"
    
    # Add failure to report
    echo "## Step 1: $STEP1_NAME" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Status:** ❌ Failed" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    
    exit 1
fi
echo

In [ ]:
#!/bin/bash

# Execute Step 2: Use output from Step 1
echo "=== Executing Step 2 ==="

# Extract step 2 details
STEP2_NAME=$(awk '/- name:/&&++count==2{print $3; exit}' "$pipeline_file" | tr -d '"')
STEP2_PROMPT=$(awk '/- name:/&&++count==2,/- name:|^pipeline:/{if(/prompt:/) {sub(/.*prompt: *"?/,""); sub(/"?$/,""); print; exit}}' "$pipeline_file")
STEP2_MODEL=$(awk '/- name:/&&++count==2,/- name:|^pipeline:/{if(/model:/) {sub(/.*model: *"?/,""); sub(/"?$/,""); print; exit}}' "$pipeline_file")

echo "Step Name: $STEP2_NAME"
echo "Model: $STEP2_MODEL"
echo "Prompt: $STEP2_PROMPT"
echo

# Prepare context-aware prompt
CONTEXT_PROMPT="Context from previous step:\n\n$(cat "$STEP1_OUTPUT")\n\n---\n\nNew task: $STEP2_PROMPT"
STEP2_OUTPUT="$FULL_OUTPUT_DIR/step2_${STEP2_NAME}.md"

echo "Executing OpenCode for step 2 with context from step 1..."

if echo -e "$CONTEXT_PROMPT" | opencode --model "$STEP2_MODEL" > "$STEP2_OUTPUT" 2>&1; then
    echo "✅ Step 2 completed successfully"
    echo "Output saved to: $STEP2_OUTPUT"
    
    # Add to pipeline report
    echo "## Step 2: $STEP2_NAME" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Status:** ✅ Completed" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Model:** $STEP2_MODEL" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Output:** [step2_${STEP2_NAME}.md](step2_${STEP2_NAME}.md)" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    
    # Show preview
    echo "--- Step 2 Output Preview ---"
    head -20 "$STEP2_OUTPUT"
    echo "--- End Preview ---"
else
    echo "❌ Step 2 failed"
    echo "Error output:"
    cat "$STEP2_OUTPUT"
    
    # Add failure to report
    echo "## Step 2: $STEP2_NAME" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "**Status:** ❌ Failed" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    
    exit 1
fi
echo

In [ ]:
#!/bin/bash

# Pipeline completion and summary
echo "=== Pipeline Execution Summary ==="

# Create final summary
echo "## Pipeline Summary" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Execution completed at:** $(date)" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Total steps:** 2 (demonstration - full pipeline would continue)" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "**Success rate:** 100%" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
echo "### Output Files" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
for file in "$FULL_OUTPUT_DIR"/*.md; do
    if [ "$file" != "$FULL_OUTPUT_DIR/pipeline_report.md" ]; then
        filename=$(basename "$file")
        echo "- [$filename]($filename)" >> "$FULL_OUTPUT_DIR/pipeline_report.md"
    fi
done

echo "Pipeline execution completed!"
echo "Results available in: $FULL_OUTPUT_DIR"
echo 
echo "Files generated:"
ls -la "$FULL_OUTPUT_DIR"
echo

echo "Pipeline report:"
cat "$FULL_OUTPUT_DIR/pipeline_report.md"
echo

echo "✅ Multi-step pipeline execution completed successfully!"
echo "📁 Output directory: $FULL_OUTPUT_DIR"
echo "📋 Session ID: $session_id"